In [2]:
import requests
import os  # Importation de la bibliothèque os

def call_chatgpt(prompt):
    api_key = os.getenv('OPENAI_API_KEY')  # Lecture de la clé API depuis une variable d'environnement
    if not api_key:
        raise ValueError("API key is not set. Please set the OPENAI_API_KEY environment variable.")

    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    payload = {
        'model': 'gpt-3.5-turbo',  # Modification du modèle à gpt-3.5-turbo
        'messages': [{
            'role': 'user',
            'content': prompt
        }]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error {response.status_code}: {response.text}"

# Test de la fonction
prompt = "What is the capital of Turkey?"
result = call_chatgpt(prompt)
print(result)


Ankara


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


data_path = "C:/Users/Abdi/Desktop/data/raw/meteo_idaweb.csv"
data = pd.read_csv(data_path)


data_info = data.info()
data_head = data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4711680 entries, 0 to 4711679
Data columns (total 19 columns):
 #   Column         Dtype  
---  ------         -----  
 0   stn            object 
 1   xchills0       float64
 2   gre000z0       float64
 3   prestas0       float64
 4   tre200s0       float64
 5   rre150z0       float64
 6   ure200s0       float64
 7   sre000z0       float64
 8   fu3010z0       float64
 9   dkl010z0       float64
 10  year           float64
 11  day_of_year    int64  
 12  minute_of_day  int64  
 13  name           object 
 14  longitude      float64
 15  latitude       float64
 16  km_longitude   float64
 17  km_latitude    float64
 18  altitude       float64
dtypes: float64(15), int64(2), object(2)
memory usage: 683.0+ MB


In [4]:
# Renaming columns based on user input to match the more descriptive names provided
data.rename(columns={
    'stn': 'station_code',
    'xchills0': 'wind_chill',
    'gre000z0': 'solar_radiation',
    'prestas0': 'atmospheric_pressure',
    'tre200s0': 'temperature_2m',
    'rre150z0': 'precipitation_1h',
    'ure200s0': 'humidity_2m',
    'sre000z0': 'snow_depth',
    'fu3010z0': 'wind_speed_10m',
    'dkl010z0': 'wind_direction_10m',
    'name': 'station_name',
    'km_longitude': 'longitude_km',
    'km_latitude': 'latitude_km'
}, inplace=True)

# Extracting data from the first row with new column names
first_row_updated = data.iloc[0]

# Formatting the extracted data into a string
formatted_data_updated = (
    f"Le jour {first_row_updated['day_of_year']} de l'année {first_row_updated['year']}, à {first_row_updated['minute_of_day']} minutes après minuit, "
    f"la station météorologique {first_row_updated['station_name']} (code {first_row_updated['station_code']}) située à une altitude de {first_row_updated['altitude']} mètres, "
    f"a enregistré une température de l'air de {first_row_updated['temperature_2m']}°C, une température ressentie de {first_row_updated['wind_chill']}°C, "
    f"une pression atmosphérique de {first_row_updated['atmospheric_pressure']} hPa, une humidité de {first_row_updated['humidity_2m']}%, "
    f"des précipitations de {first_row_updated['precipitation_1h']} mm, et une profondeur de neige de {first_row_updated['snow_depth']} cm. "
    f"La vitesse du vent était de {first_row_updated['wind_speed_10m']} m/s venant de la direction {first_row_updated['wind_direction_10m']} degrés."
)

# Constructing the full prompt with an explanatory comment for the LLM
prompt_updated = (
    f"{formatted_data_updated} Sur la base de ces données, quelle serait votre prédiction météorologique pour le reste de la journée? "
    "Veuillez considérer les modèles habituels de changements météorologiques pour cette période de l'année et cette localisation."
)

formatted_data_updated, prompt_updated


("Le jour 1 de l'année 2020.0, à 0 minutes après minuit, la station météorologique Adelboden (code ABO) située à une altitude de 1321.0 mètres, a enregistré une température de l'air de 2.7°C, une température ressentie de 2.7°C, une pression atmosphérique de 879.1 hPa, une humidité de 55.9%, des précipitations de 0.0 mm, et une profondeur de neige de 0.0 cm. La vitesse du vent était de 2.2 m/s venant de la direction 228.0 degrés.",
 "Le jour 1 de l'année 2020.0, à 0 minutes après minuit, la station météorologique Adelboden (code ABO) située à une altitude de 1321.0 mètres, a enregistré une température de l'air de 2.7°C, une température ressentie de 2.7°C, une pression atmosphérique de 879.1 hPa, une humidité de 55.9%, des précipitations de 0.0 mm, et une profondeur de neige de 0.0 cm. La vitesse du vent était de 2.2 m/s venant de la direction 228.0 degrés. Sur la base de ces données, quelle serait votre prédiction météorologique pour le reste de la journée? Veuillez considérer les modèl

Extraire toutes les données et les formater en chaînes de caractères pour chaque enregistrement

In [5]:
# Function to format each row into a descriptive string
def format_weather_data(row):
    return (
        f"Le jour {row['day_of_year']} de l'année {row['year']}, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

# Apply the formatting function to each row in the dataset
formatted_weather_data = data.apply(format_weather_data, axis=1)

# Display the first few formatted entries
formatted_weather_data.head()


0    Le jour 1 de l'année 2020.0, à 0 minutes après...
1    Le jour 1 de l'année 2020.0, à 10 minutes aprè...
2    Le jour 1 de l'année 2020.0, à 20 minutes aprè...
3    Le jour 1 de l'année 2020.0, à 30 minutes aprè...
4    Le jour 1 de l'année 2020.0, à 40 minutes aprè...
dtype: object

Creation de prompt
Inclure un commentaire ou une question spécifique qui oriente le modèle sur ce que nous souhaitons obtenir comme information ou analyse

In [6]:
# Supposons que 'formatted_weather_data' est votre liste de chaînes formatées.
weather_data_strings = formatted_weather_data.tolist()

# Concaténation des données pour les inclure dans le prompt
combined_data = " ".join(weather_data_strings)  # Utiliser seulement les 10 premières pour l'exemple

# Création du prompt
prompt = f"Voici les conditions météorologiques enregistrées à différentes heures pour tous les stations: {combined_data} Sur la base de ces informations, il ya combien des stations?"
result = call_chatgpt(prompt)
print(result)
print(prompt)


OverflowError: string longer than 2147483647 bytes

In [17]:
import pandas as pd

# Charger vos données dans un DataFrame (supposons que cela soit déjà fait et que le DataFrame s'appelle 'data')

# Filtrer les données pour la station 'Sion'
sion_data = data[data['station_name'] == 'Sion']

# Formater chaque ligne de données filtrées
def format_weather_data(row):
    return (
        f"Le jour {row['day_of_year']} de l'année {row['year']}, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

formatted_sion_data = sion_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_data = " ".join(formatted_sion_data.tolist())

# Créer le prompt
prompt = (f"Voici les conditions météorologiques enregistrées à différentes heures pour la station Sion: {combined_sion_data} Sur la base de ces informations, pouvez-vous fournir une analyse des tendances météorologiques observées et prédire les conditions pour le reste de la semaine?")

# Utilisation du prompt pour interroger un modèle de langage
result = call_chatgpt(prompt)
print(result)


Error 429: {
    "error": {
        "message": "Request too large for gpt-3.5-turbo in organization org-1E9n2SMyY1aXv1qhmJC660uG on tokens per min (TPM): Limit 160000, Requested 17676569. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.",
        "type": "tokens",
        "param": null,
        "code": "rate_limit_exceeded"
    }
}



In [19]:


# Filtrer les données pour la station 'Sion'

sion_2020_data = data[(data['station_name'] == 'Sion') & (data['year'] == 2020)]


# Formater chaque ligne de données filtrées
def format_weather_data(row):
    return (
        f"Le jour {row['day_of_year']} de l'année {row['year']}, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

formatted_sion_data = sion_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_data = " ".join(formatted_sion_data.tolist())

# Créer le prompt
prompt = (f"Voici les conditions météorologiques enregistrées à différentes heures pour la station Sion: {combined_sion_data} Sur la base de ces informations, pouvez-vous fournir une analyse des tendances météorologiques observées et prédire les conditions pour le reste de la semaine?")

# Utilisation du prompt pour interroger un modèle de langage
result = call_chatgpt(prompt)
print(result)


Error 429: {
    "error": {
        "message": "Request too large for gpt-3.5-turbo in organization org-1E9n2SMyY1aXv1qhmJC660uG on tokens per min (TPM): Limit 160000, Requested 17676569. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.",
        "type": "tokens",
        "param": null,
        "code": "rate_limit_exceeded"
    }
}



In [20]:
data.columns

Index(['station_code', 'wind_chill', 'solar_radiation', 'atmospheric_pressure',
       'temperature_2m', 'precipitation_1h', 'humidity_2m', 'snow_depth',
       'wind_speed_10m', 'wind_direction_10m', 'year', 'day_of_year',
       'minute_of_day', 'station_name', 'longitude', 'latitude',
       'longitude_km', 'latitude_km', 'altitude'],
      dtype='object')

In [22]:
import pandas as pd

# Chargement de votre DataFrame
# data = pd.read_your_data_method()

# Assurez-vous que 'year' et 'day_of_year' sont du bon type
data['year'] = data['year'].astype(int)
data['day_of_year'] = data['day_of_year'].astype(int)

# Vérifiez les valeurs max et min pour s'assurer qu'elles sont dans le range attendu
print('Year range:', data['year'].min(), data['year'].max())
print('Day of year range:', data['day_of_year'].min(), data['day_of_year'].max())

# Convertir 'year' et 'day_of_year' en une colonne datetime
try:
    data['date'] = pd.to_datetime(data['year'].astype(str) + data['day_of_year'].astype(str).str.zfill(3), format='%Y%j')
    data['month'] = data['date'].dt.month
    print(data[['date', 'month']].head())
except ValueError as e:
    print("Error converting to datetime:", e)


Year range: 2020 2022
Day of year range: 1 366
        date  month
0 2020-01-01      1
1 2020-01-01      1
2 2020-01-01      1
3 2020-01-01      1
4 2020-01-01      1


In [23]:
import pandas as pd

# Supposons que votre DataFrame s'appelle 'data'

# Convertir 'year' et 'day_of_year' en une colonne datetime
data['date'] = pd.to_datetime(data['year'].astype(str) + data['day_of_year'].astype(str), format='%Y%j')

# Extraire le mois de la nouvelle colonne de date
data['month'] = data['date'].dt.month

# Maintenant, vous pouvez vérifier les premières lignes pour voir la nouvelle colonne 'month'
print(data[['date', 'month']].head())


        date  month
0 2020-01-01      1
1 2020-01-01      1
2 2020-01-01      1
3 2020-01-01      1
4 2020-01-01      1


Script Python pour Filtrer les Données pour "Sion" en Juillet 2020



In [25]:
# Filtrer pour juillet 2020 et la station Sion
sion_juillet_2020_data = data[(data['station_name'] == 'Sion') & (data['month'] == 7) & (data['year'] == 2020)]

# Formatter les données filtrées pour un modèle de langage
def format_weather_data(row):
    return (
        f"Le jour {row['day_of_year']} de l'année {row['year']}, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

formatted_sion_juillet_2020_data = sion_juillet_2020_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_juillet_2020_data = " ".join(formatted_sion_juillet_2020_data.tolist())

# Créer le prompt
prompt = (f"Voici les conditions météorologiques enregistrées à différentes heures pour la station Sion en juillet 2020: {combined_sion_juillet_2020_data} Sur la base de ces informations, pouvez-vous fournir une analyse des tendances météorologiques observées et prédire les conditions pour le reste de l'été?")

# Utilisation du prompt pour interroger un modèle de langage
result = call_chatgpt(prompt)
print(result)


Error 429: {
    "error": {
        "message": "Request too large for gpt-3.5-turbo in organization org-1E9n2SMyY1aXv1qhmJC660uG on tokens per min (TPM): Limit 160000, Requested 499832. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.",
        "type": "tokens",
        "param": null,
        "code": "rate_limit_exceeded"
    }
}



In [26]:
# Filtrer pour le 1er juillet 2020 et la station Sion
sion_1_juillet_2020_data = data[(data['station_name'] == 'Sion') & (data['month'] == 7) & (data['year'] == 2020) & (data['day_of_year'] == 183)]

# Formatter les données filtrées pour un modèle de langage
def format_weather_data(row):
    return (
        f"Le 1er juillet 2020, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

formatted_sion_1_juillet_2020_data = sion_1_juillet_2020_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_1_juillet_2020_data = " ".join(formatted_sion_1_juillet_2020_data.tolist())

# Créer le prompt
prompt = f"Voici les conditions météorologiques enregistrées à différentes heures pour la station Sion le 1er juillet 2020: {combined_sion_1_juillet_2020_data} Sur la base de ces informations, pouvez-vous fournir une analyse des tendances météorologiques observées et prédire les conditions pour le reste de l'été?"

# Utilisation du prompt pour interroger un modèle de langage
result = call_chatgpt(prompt)
print(result)


Error 400: {
  "error": {
    "message": "This model's maximum context length is 16385 tokens. However, your messages resulted in 22150 tokens. Please reduce the length of the messages.",
    "type": "invalid_request_error",
    "param": "messages",
    "code": "context_length_exceeded"
  }
}



In [27]:
# Filtrer pour le 1er juillet 2020 et la station Sion
sion_1_juillet_2020_data = data[(data['station_name'] == 'Sion') & (data['month'] == 7) & (data['year'] == 2020) & (data['day_of_year'] == 183)]

# Formatter les données filtrées pour un modèle de langage
def format_weather_data(row):
    return (
        f"Le 1er juillet 2020, à {row['minute_of_day']} minutes après minuit, "
        f"la station météorologique {row['station_name']} (code {row['station_code']}) située à une altitude de {row['altitude']} mètres, "
        f"a enregistré une température de l'air de {row['temperature_2m']}°C, une température ressentie de {row['wind_chill']}°C, "
        f"une pression atmosphérique de {row['atmospheric_pressure']} hPa, une humidité de {row['humidity_2m']}%, "
        f"des précipitations de {row['precipitation_1h']} mm, et une profondeur de neige de {row['snow_depth']} cm. "
        f"La vitesse du vent était de {row['wind_speed_10m']} m/s venant de la direction {row['wind_direction_10m']} degrés."
    )

formatted_sion_1_juillet_2020_data = sion_1_juillet_2020_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_1_juillet_2020_data = " ".join(formatted_sion_1_juillet_2020_data.tolist())

# Créer le prompt
prompt = f"Voici les conditions météorologiques enregistrées à différentes heures pour la station Sion le 1er juillet 2020: {combined_sion_1_juillet_2020_data} Sur la base de ces informations, pouvez-vous fournir une analyse des tendances météorologiques observées et prédire les conditions pour le reste de l'été?"

# Utilisation du prompt pour interroger un modèle de langage
result = call_chatgpt(prompt)
print(result)

Error 400: {
  "error": {
    "message": "This model's maximum context length is 16385 tokens. However, your messages resulted in 22150 tokens. Please reduce the length of the messages.",
    "type": "invalid_request_error",
    "param": "messages",
    "code": "context_length_exceeded"
  }
}



Script Python pour Extraire les Données Essentielles

In [28]:
# Filtrer pour le 1er juillet 2020 et la station Sion
sion_1_juillet_2020_data = data[(data['station_name'] == 'Sion') & (data['month'] == 7) & (data['year'] == 2020) & (data['day_of_year'] == 183)]

# Formatter les données filtrées pour un modèle de langage en se concentrant uniquement sur les variables essentielles
def format_weather_data(row):
    return (
        f"À {row['minute_of_day']} minutes après minuit, température: {row['temperature_2m']}°C, "
        f"précipitations: {row['precipitation_1h']} mm, humidité: {row['humidity_2m']}%, "
        f"rayonnement solaire: {row['solar_radiation']} W/m²."
    )

formatted_sion_1_juillet_2020_data = sion_1_juillet_2020_data.apply(format_weather_data, axis=1)

# Concaténer les données formatées
combined_sion_1_juillet_2020_data = " ".join(formatted_sion_1_juillet_2020_data.tolist())

# Créer le prompt
prompt = f"Voici les données météorologiques essentielles pour la station Sion le 1er juillet 2020 à différentes heures: {combined_sion_1_juillet_2020_data} Sur la base de ces informations, pouvez-vous analyser les conditions et fournir des recommandations pour les activités extérieures ?"

# Utilisation du prompt pour interroger un modèle de langage
print(prompt)


Voici les données météorologiques essentielles pour la station Sion le 1er juillet 2020 à différentes heures: À 0 minutes après minuit, température: 18.3°C, précipitations: 0.0 mm, humidité: 76.1%, rayonnement solaire: 0.0 W/m². À 10 minutes après minuit, température: 17.3°C, précipitations: 0.0 mm, humidité: 82.8%, rayonnement solaire: 0.0 W/m². À 20 minutes après minuit, température: 17.4°C, précipitations: 0.0 mm, humidité: 81.7%, rayonnement solaire: 0.0 W/m². À 30 minutes après minuit, température: 17.1°C, précipitations: 0.0 mm, humidité: 81.2%, rayonnement solaire: 0.0 W/m². À 40 minutes après minuit, température: 16.7°C, précipitations: 0.0 mm, humidité: 82.3%, rayonnement solaire: 0.0 W/m². À 50 minutes après minuit, température: 15.4°C, précipitations: 0.0 mm, humidité: 91.6%, rayonnement solaire: 0.0 W/m². À 60 minutes après minuit, température: 17.9°C, précipitations: 0.0 mm, humidité: 76.8%, rayonnement solaire: 0.0 W/m². À 70 minutes après minuit, température: 15.9°C, pré

In [29]:
result = call_chatgpt(prompt)
print(result)

Les données météorologiques montrent que la température à Sion varie entre 14.6°C et 31.1°C au cours de la journée du 1er juillet 2020, avec un taux d'humidité allant de 37.4% à 97.5%. Il n'y a pas de précipitations et le rayonnement solaire varie de 0.0 W/m² à 1000 W/m².

En analysant ces informations, voici quelques recommandations pour les activités extérieures:

- Le matin et en début d'après-midi, il est recommandé de profiter du temps ensoleillé mais de rester hydraté et de se protéger du soleil en utilisant de la crème solaire et en portant un chapeau.
- En milieu d'après-midi, lorsque la température est la plus élevée, il est conseillé de rechercher des endroits ombragés pour éviter les coups de chaleur et de maintenir une hydratation adéquate.
- En fin d'après-midi et en soirée, lorsque la température commence à baisser, c'est un bon moment pour les activités extérieures plus intenses comme le sport ou une promenade.
- Il est important de rester informé des conditions météorol